# **Justification of the project**

The purpose of this Notebook is to generate a structured and comprehensive JSON-based profile of any dataset provided at the path `data/data.csv`, serving as an automated and reusable data profiling stage.

The notebook performs a dataset-agnostic exploratory analysis, producing both global metadata and detailed per-column summaries without relying on domain-specific assumptions. At the dataset level, the analysis captures information such as dimensionality, memory usage, duplicated rows, and overall missing-value ratios. At the column level, it extracts data types, cardinality, missing-value statistics, and extended descriptive measures adapted to numerical and categorical variables, while explicitly handling index and identifier columns to prevent their misuse as analytical features.

Beyond descriptive profiling, the generated JSON is designed to act as an intermediate artifact for automated reasoning. In a subsequent step, implemented in a separate Python script, this JSON output is used as input to a locally deployed Large Language Model (LLM), for example via Ollama. The LLM is queried to generate a high-level analytical summary of the dataset, extracting conclusions and suggesting potential actions based on the observed statistics and quality indicators.

This reasoning step enables the detection and interpretation of relevant patterns that are not explicitly encoded as rules, such as identifying potential outliers when the difference between upper quartiles and maximum values is unusually large, highlighting columns that may require scaling, transformation, or cleaning, and summarizing overall data quality concerns in natural language.

The combination of automated statistical profiling and LLM-based interpretative analysis results in a hybrid pipeline that bridges low-level data inspection and high-level analytical insight. The final outcome is a reproducible, extensible workflow that supports data quality assessment, feature engineering decisions, and preliminary analytical conclusions in a form suitable for both technical pipelines and human interpretation.


In [1]:
import pandas as pd
import json
import datetime

# Define file paths
TEST_DATA_PATH = "data/data.csv"
OUTPUT_PATH = "processed_data.json"

# list of strings to be considered as NaN values
nan_vals = ["", "NaN", "nan", "NULL", "null", "N/A", "n/a", "NA", "na", "None", "none", "-", "?"]
# threshold for considering a column as having too many NaN values
nan_interval = 0.05 # 5%

# read the CSV file into a DataFrame, treating specified strings as NaN
df = pd.read_csv(TEST_DATA_PATH, na_values = nan_vals)
df.set_index(df.columns[0], inplace=True)

# calculate the number of NaN values per column
nan_per_column = df.isna().sum()

In [2]:
# =========================
# REPORT INITIAL STRUCTURE
# =========================

report = {
    "timestamp": datetime.datetime.now().isoformat(),
    "total_rows": df.shape[0],
    "total_columns": df.shape[1],
    "columns": []
}

# =========================
# DATASET GLOBAL METADATA
# =========================

report["dataset_info"] = {
    "source_file": TEST_DATA_PATH,
    "memory_usage_mb": df.memory_usage(deep=True).sum() / 1e6,
    "duplicated_rows": int(df.duplicated().sum()),
    "percent_nan_global": float(
        df.isna().sum().sum() / (df.shape[0] * df.shape[1]) * 100
    )
}

report["environment"] = {
    "pandas_version": pd.__version__,
    "generated_at": datetime.datetime.now().isoformat()
}

In [3]:
# =========================
# INDEX REPORT
# =========================

index_name = df.index.name if df.index.name is not None else "__index__"

report["columns"].append(
    {
        "column_name": index_name,
        "data_type": str(df.index.dtype),
        "num_unique_values": int(pd.Index(df.index).nunique()),
        "num_nan_values": int(pd.isna(df.index).sum()),
        "percent_nan": (pd.isna(df.index).sum() / df.shape[0]) * 100,
        "is_index": True,
        "role": "index",
        "recommendation": "keep"
    }
)

In [4]:
# =========================
# COLUMN PROFILING
# =========================

for column in df.columns:

    col = df[column]
    num_nan = int(col.isna().sum())
    percent_nan = (num_nan / df.shape[0]) * 100
    num_unique = int(col.nunique(dropna=True))

    col_report = {
        "column_name": column,
        "data_type": str(col.dtype),
        "num_unique_values": num_unique,
        "num_nan_values": num_nan,
        "percent_nan": percent_nan,
        "quality_warnings": []
    }

    # =========================
    # SEMANTIC FLAGS
    # =========================

    col_report["is_constant"] = num_unique == 1
    col_report["is_sparse"] = percent_nan > (nan_interval * 100)
    col_report["is_identifier"] = (num_unique / df.shape[0]) > 0.95 and not pd.api.types.is_float_dtype(col)
    col_report["role"] = "feature"

    if col_report["is_identifier"]:
        col_report["role"] = "identifier"

    # =========================
    # NUMERIC FEATURES
    # =========================

    if pd.api.types.is_numeric_dtype(col):

        q25 = col.quantile(0.25)
        q75 = col.quantile(0.75)

        col_report.update({
            "min_value": col.min(),
            "mean": col.mean(),
            "std": col.std(),
            "25_quartile": q25,
            "median": col.median(),
            "75_quartile": q75,
            "max_value": col.max(),
            "iqr": q75 - q25
        })

        if col_report["std"] == 0 or pd.isna(col_report["std"]):
            col_report["quality_warnings"].append("zero_variance")

    # =========================
    # CATEGORICAL FEATURES
    # =========================

    elif pd.api.types.is_categorical_dtype(col) or pd.api.types.is_object_dtype(col):

        value_counts = col.value_counts(dropna=True)

        if not value_counts.empty:
            col_report["most_common"] = value_counts.index[0]
            col_report["most_common_freq"] = int(value_counts.iloc[0])
            col_report["top_3_values"] = value_counts.head(3).to_dict()
            col_report["num_rare_categories"] = int((value_counts == 1).sum())

    # =========================
    # QUALITY WARNINGS
    # =========================

    if percent_nan > 50:
        col_report["quality_warnings"].append("high_missing_ratio")

    if col_report["is_constant"]:
        col_report["quality_warnings"].append("constant_column")

    # =========================
    # RECOMMENDATIONS
    # =========================

    if col_report["is_identifier"]:
        col_report["recommendation"] = "exclude_from_model"
    elif percent_nan > 50:
        col_report["recommendation"] = "consider_drop"
    elif pd.api.types.is_numeric_dtype(col):
        col_report["recommendation"] = "consider_scaling"
    else:
        col_report["recommendation"] = "consider_encoding"

    report["columns"].append(col_report)

# =========================
# GLOBAL QUALITY SUMMARY
# =========================

report["quality_summary"] = {
    "columns_high_nan": [
        c["column_name"]
        for c in report["columns"]
        if c.get("percent_nan", 0) > 50
    ],
    "constant_columns": [
        c["column_name"]
        for c in report["columns"]
        if c.get("is_constant")
    ],
    "identifier_columns": [
        c["column_name"]
        for c in report["columns"]
        if c.get("is_identifier")
    ]
}


C:\Users\tonac\AppData\Local\Temp\ipykernel_20292\3262591538.py:60: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(col) or pd.api.types.is_object_dtype(col):


In [5]:
# =========================
# JSON EXPORT
# =========================

report_json = json.dumps(report, indent=4)

with open(OUTPUT_PATH, "w") as f:
    f.write(report_json)